In [116]:
horizon, lookback = 7,15
items = L(np.arange(-5,100)[None,:],np.arange(500,550)[None,:],np.arange(-110,-56)[None,:]).map(tensor)
data = TSDataLoaders.from_items(items, horizon = horizon, lookback=lookback, step=1, after_batch = NormalizeTS()
                               )

mdl = NBeatsNet(device = data.train.device, stack_types=('trend','seaonality'), horizon=horizon, lookback=lookback)
loss_func = F.mse_loss
learn = Learner(data, mdl, loss_func=loss_func, opt_func= Adam, 
                cbs=L(NBeatsTrainer())
               )

Need to pad 1/3 time series due to length.
Train:60; Valid: 48; Test 3


In [117]:
xb = data.one_batch()[0]

In [120]:
def example_forward_hook(m,i,o): 
    print( i.shape, o.shape)
hook = mdl.register_forward_hook(example_forward_hook)
y = mdl(xb)
hook.remove()

TypeError: object of type 'NBeatsNet' has no len()

In [109]:
lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, nn.ReLU)
def find_modules(m, cond):
    if cond(m): return [m]
    return sum([find_modules(o,cond) for o in m.children()], [])
mods = find_modules(mdl, lambda o: isinstance(o,lin_layers))
mods

[Linear(in_features=15, out_features=200, bias=False),
 ReLU(inplace=True),
 Linear(in_features=200, out_features=100, bias=False),
 Linear(in_features=100, out_features=1, bias=False),
 Linear(in_features=100, out_features=3, bias=False),
 Linear(in_features=15, out_features=200, bias=False),
 ReLU(inplace=True),
 Linear(in_features=200, out_features=100, bias=False),
 Linear(in_features=100, out_features=1, bias=False),
 Linear(in_features=100, out_features=3, bias=False),
 Linear(in_features=15, out_features=200, bias=False),
 ReLU(inplace=True),
 Linear(in_features=200, out_features=100, bias=False),
 Linear(in_features=100, out_features=1, bias=False),
 Linear(in_features=100, out_features=3, bias=False),
 Linear(in_features=15, out_features=200, bias=False),
 ReLU(inplace=True),
 Linear(in_features=200, out_features=100, bias=False),
 Linear(in_features=100, out_features=1, bias=False),
 Linear(in_features=100, out_features=3, bias=False),
 LinearD(in_features=3, out_features=7, 

In [112]:
def append_stat(hook, mod, outp):
    print(hook,mod,outp)
    assert False
    d = outp.data
    hook.mean,hook.std = d.mean().item(),d.std().item()
    
with Hooks(mods, append_stat) as hooks:
    mdl(xb)
    for hook in hooks:
        print(hook.mean,hook.std)

TypeError: append_stat() missing 1 required positional argument: 'outp'

In [111]:
def lsuv_module(m, xb):
    h = Hook(m, append_stat)

    while mdl(xb) is not None and abs(h.mean)  > 1e-3: m.bias -= h.mean
    while mdl(xb) is not None and abs(h.std-1) > 1e-3: m.weight.data /= h.std

    h.remove()
    return h.mean,h.std

for m in mods: 
    print(lsuv_module(m, xb))

TypeError: append_stat() missing 1 required positional argument: 'outp'